In [ ]:
import numpy as np

from fbpinns.domains import RectangularDomainND
from fbpinns.problems import Gravity
from fbpinns.decompositions import MultilevelRectangularDecompositionND
from fbpinns.networks import FCN
from fbpinns.schedulers import AllActiveSchedulerND
from fbpinns.constants import Constants, get_subdomain_ws
from fbpinns.trainers import FBPINNTrainer

XMIN, XMAX, YMIN, YMAX = 0, 1, 0, 1

w = 3.9
# subdomain_xss = [[np.array([0.5]),np.array([0.5])]]
    # [[np.linspace(0,1,n_),np.linspace(0,1,n_)] for n_ in (5,)] +\
    
# subdomain_wss = [[np.array([1.]),np.array([1.])]]
    # [get_subdomain_ws(subdomain_xss[1], w)] +\

subdomain_xss = [[np.array([0.5]),np.array([0.5])]] + [[np.linspace(0,1,n_),np.linspace(0,1,n_)] for n_ in (10,)]
#     # [[np.linspace(0,1,n_),np.linspace(0,1,n_)] for n_ in (5,)] +\
    
subdomain_wss = [[np.array([1.]),np.array([1.])]] + [get_subdomain_ws(subdomain_xss[1], w)]
#     # [get_subdomain_ws(subdomain_xss[1], w)] +\
    

c = Constants(
    domain=RectangularDomainND,
    domain_init_kwargs = dict(
        xmin=np.array([0.,0.]),
        xmax=np.array([1.,1.])
        ),
    problem=Gravity,
    problem_init_kwargs = dict(sd=0.1),
    decomposition=MultilevelRectangularDecompositionND,
    decomposition_init_kwargs=dict(
        subdomain_xss=subdomain_xss,
        subdomain_wss=subdomain_wss,
        unnorm=(0.,3.),
    ),
    network=FCN,
    network_init_kwargs = dict(
        layer_sizes = (2, 32, 12, 1),
    ),
    optimiser_kwargs=dict(learning_rate=5e-4),
    scheduler = AllActiveSchedulerND,
    ns=((100,100),),
    n_test=(100,100),
    n_steps=100000,
    clear_output=True,
)

run = FBPINNTrainer(c)
all_params = run.train()

[INFO] 2024-12-11 20:47:04 - <fbpinns.constants.Constants object at 0x7fb8d8f6e660>
run: test
domain: <class 'fbpinns.domains.RectangularDomainND'>
domain_init_kwargs: {'xmin': array([0., 0.]), 'xmax': array([1., 1.])}
problem: <class 'fbpinns.problems.Gravity'>
problem_init_kwargs: {'sd': 0.1}
decomposition: <class 'fbpinns.decompositions.MultilevelRectangularDecompositionND'>
decomposition_init_kwargs: {'subdomain_xss': [[array([0.5]), array([0.5])], [array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
       0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ]), array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
       0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ])]], 'subdomain_wss': [[array([1.]), array([1.])], [array([0.43333333, 0.43333333, 0.43333333, 0.43333333, 0.43333333,
       0.43333333, 0.43333333, 0.43333333, 0.43333333, 0.43333333]), array([0.43333333, 0.43333333, 0.43333333, 0.43333333, 0.43333333,
       0.43333333,